In [1]:
import time
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
csv_file_paths = [
    './hard_hhhss_783.csv',
    './hard_sssod_ws_781.csv',
    './hard_sss_777.csv',
    './ocr_hr_jh_705.csv',
    './deep_bs_700.csv'
    ]

In [3]:
def csv_to_one_hots(csv_file_path):
    csv_df = pd.read_csv(csv_file_path)
    pred_strs = csv_df["PredictionString"]
    
    image_pred_one_hots_list = list()
    for pred_str in pred_strs:
        pred_ints = [int(pred) for pred in pred_str.split(' ')]
        pred_one_hots = [np.eye(11)[pred_int] for pred_int in pred_ints]
        image_pred_one_hots_list.append(pred_one_hots)
        
    return image_pred_one_hots_list

In [4]:
images_pred_arrays = list()
for csv_file_path in csv_file_paths:
    print(f'Loading {csv_file_path} ...') 
    csv_df = pd.read_csv(csv_file_path)
    pred_strs = csv_df["PredictionString"]
    
    image_pred_one_hots_list = list()
    for pred_str in tqdm(pred_strs):
        pred_ints = [int(pred) for pred in pred_str.split(' ')]
        pred_one_hots = [np.eye(11, dtype=np.uint8)[pred_int] for pred_int in pred_ints]
        image_pred_one_hots_list.append(pred_one_hots)
    
    images_pred_arrays.append(np.array(image_pred_one_hots_list))
    
images_pred_4d_array = np.array(images_pred_arrays)

del images_pred_arrays
    
print(images_pred_4d_array.shape)
print(images_pred_4d_array.dtype)


Loading ./hard_hhhss_783.csv ...


100%|██████████| 819/819 [01:53<00:00,  7.31it/s]


Loading ./hard_sssod_ws_781.csv ...


100%|██████████| 819/819 [01:51<00:00,  7.52it/s]


Loading ./hard_sss_777.csv ...


...

Loading ./ocr_hr_jh_705.csv ...


100%|██████████| 819/819 [01:52<00:00,  7.52it/s]


Loading ./deep_bs_700.csv ...


100%|██████████| 819/819 [01:51<00:00,  7.53it/s]


(5, 819, 65536, 11)
uint8


In [5]:
count_pred_array = np.sum(images_pred_4d_array, axis=0)
print(count_pred_array.shape)

(819, 65536, 11)


In [6]:
final_pred_array = np.argmax(count_pred_array, axis=2)
print(final_pred_array.shape)

(819, 65536)


In [7]:
submission_df = pd.DataFrame()

file_names = pd.read_csv(csv_file_paths[0], index_col=None)['image_id']

for i_img in range(len(file_names)):
    pred_str = ' '.join([str(pred_int) for pred_int in final_pred_array[i_img].tolist()])
    submission_df = submission_df.append({"image_id": file_names[i_img], "PredictionString": pred_str}, ignore_index=True)
                          
submission_df.to_csv("./hard_voting_ensemble_hhhod.csv", index=False)